### Libraries:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import pad_sequences
import tensorflow as tf
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import numpy as np
from sklearn.dummy import DummyRegressor
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense

### Warnings:

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import pandas as pd
# import io

# df = pd.read_csv(io.BytesIO(uploaded['data1_filled.csv']))
# df

### Read dataset:

In [3]:
df = pd.read_csv('data2_filled.csv')
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


### Group by Study_Folder:

In [4]:
ispitanci_X = []
ispitanici_y = []
for s, df_s in df.groupby(['Study_Folder']):
    train_mask = df_s['Stimuli'] < 10
    valid_mask = (df_s['Stimuli'] >= 10) & (df_s['Stimuli'] <= 11)
    test_mask = df_s['Stimuli'] > 11
    features = df_s.columns[4:13]
    ys = df_s.columns[13:15]
    X_tr, X_v, X_te = df_s[train_mask][features], df_s[valid_mask][features], df_s[test_mask][features]
    y_tr, y_v, y_te = df_s[train_mask][ys], df_s[valid_mask][ys], df_s[test_mask][ys]

    ispitanci_X.append([X_tr, X_v, X_te])
    ispitanici_y.append([y_tr, y_v, y_te])

### Metrics:

In [5]:
metric_dict = {"MAE":mean_absolute_error, "MSE":mean_squared_error, "R2": r2_score}

### Log table:

In [6]:
log_table_total =[]

### Dummy Regressor:

In [7]:
dummy_regr = DummyRegressor(strategy="mean")
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go modelot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    dummy_regr.fit(X_tr, y_tr)
    y_pred_dr=dummy_regr.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['DummyRegressor', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_dr)])

100%|██████████| 23/23 [00:00<00:00, 193.73it/s]


In [8]:
n_steps = 3
input_features = 9
n_units = 50

### Function for creating sequences:

In [9]:
def create_sequences(data_X, data_y, n_steps_in=3): # many to many:
    m,n = [],[]
    for i in range(data_X.shape[0]-n_steps_in):
        m.append(data_X.values[i:i+n_steps]); n.append(data_y.values[i+n_steps])
    return np.array(m), np.array(n)

### SimpleRNN + LSTM + GRU:

In [10]:
model_2 = Sequential()
model_2.add(SimpleRNN(200, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_2.add(LSTM(n_units, activation = 'relu', return_sequences = True))
model_2.add(GRU(n_units, activation = 'tanh'))
model_2.add(Dense(2, activation = 'linear'))
model_2.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3, 200)            42000     
                                                                 
 lstm (LSTM)                 (None, 3, 50)             50200     
                                                                 
 gru (GRU)                   (None, 50)                15300     
                                                                 
 dense (Dense)               (None, 2)                 102       
                                                                 
Total params: 107,602
Trainable params: 107,602
Non-trainable params: 0
_________________________________________________________________


In [11]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_2.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_2.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['SimpleRNN+LSTM+GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/23 [00:00<?, ?it/s]

22/22 [==============================] - 1s 3ms/step


  4%|▍         | 1/23 [00:14<05:19, 14.52s/it]

22/22 [==============================] - 0s 6ms/step


  9%|▊         | 2/23 [00:16<02:33,  7.29s/it]

...

 91%|█████████▏| 21/23 [01:52<00:11,  5.58s/it]

18/18 [==============================] - 0s 3ms/step


 96%|█████████▌| 22/23 [01:54<00:04,  4.69s/it]

43/43 [==============================] - 0s 5ms/step


100%|██████████| 23/23 [02:00<00:00,  5.25s/it]


### Many to many (3 rows for train, predict on 2):

In [12]:
n_steps = 3
input_features = 9
n_steps_out=2
n_units = 50

In [13]:
def create_sequences_1(data_X, data_y, n_steps_in=3,n_steps_out=2): # many to many:
    m,n = [],[]
    for i in range(data_X.shape[0]-n_steps_in-n_steps_out+1):
        m.append(data_X.values[i:i+n_steps])
        new_y=data_y.values[i+n_steps:i+n_steps_in+n_steps_out].flatten()
        n.append(new_y)
    return np.array(m), np.array(n)

In [14]:
model_3 = Sequential()
model_3.add(SimpleRNN(n_units, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_3.add(LSTM(100, activation = 'relu', return_sequences = True))
model_3.add(GRU(n_units, activation = 'relu'))
model_3.add(Dense(n_steps_out*2, activation = 'linear'))
model_3.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 3, 50)             3000      
                                                                 
 lstm_1 (LSTM)               (None, 3, 100)            60400     
                                                                 
 gru_1 (GRU)                 (None, 50)                22800     
                                                                 
 dense_1 (Dense)             (None, 4)                 204       
                                                                 
Total params: 86,404
Trainable params: 86,404
Non-trainable params: 0
_________________________________________________________________


In [15]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go modelot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences_1(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences_1(X_v, y_v, n_steps)
    X_te, y_te = create_sequences_1(X_te, y_te, n_steps)


    model_3.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_3.predict(X_te)

    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['SimpleRNN+LSTM+GRU_many_to_many', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/23 [00:00<?, ?it/s]

22/22 [==============================] - 1s 5ms/step


  4%|▍         | 1/23 [00:08<03:08,  8.57s/it]

22/22 [==============================] - 0s 5ms/step


  9%|▊         | 2/23 [00:16<02:56,  8.42s/it]

...

 91%|█████████▏| 21/23 [01:49<00:09,  4.68s/it]

18/18 [==============================] - 0s 2ms/step


 96%|█████████▌| 22/23 [01:51<00:03,  3.77s/it]

43/43 [==============================] - 0s 6ms/step


100%|██████████| 23/23 [01:57<00:00,  5.12s/it]


### Conv1D + SimpleRNN + LSTM + GRU:

In [16]:
model_4 = Sequential()
model_4.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(3, 9)))
model_4.add(SimpleRNN(n_units, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_4.add(LSTM(100, activation = 'relu', return_sequences = True))
model_4.add(GRU(n_units, activation = 'sigmoid'))
model_4.add(Dense(2, activation = 'linear'))
model_4.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_4.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 16)             448       
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 1, 50)             3350      
                                                                 
 lstm_2 (LSTM)               (None, 1, 100)            60400     
                                                                 
 gru_2 (GRU)                 (None, 50)                22800     
                                                                 
 dense_2 (Dense)             (None, 2)                 102       
                                                                 
Total params: 87,100
Trainable params: 87,100
Non-trainable params: 0
_________________________________________________________________


In [17]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_4.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_4.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['Conv1D&SimpleRNN&LSTM&GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/23 [00:00<?, ?it/s]

22/22 [==============================] - 1s 2ms/step


  4%|▍         | 1/23 [00:11<04:21, 11.91s/it]

22/22 [==============================] - 0s 2ms/step


  9%|▊         | 2/23 [00:13<02:00,  5.74s/it]

...

 91%|█████████▏| 21/23 [02:01<00:09,  4.78s/it]

18/18 [==============================] - 0s 4ms/step


 96%|█████████▌| 22/23 [02:04<00:04,  4.23s/it]

43/43 [==============================] - 0s 2ms/step


100%|██████████| 23/23 [02:08<00:00,  5.59s/it]


### Conv1D + LSTM + GRU

In [18]:
model_6 = Sequential()
model_6.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(3, 9)))
model_6.add(LSTM(100, activation = 'relu',input_shape = (n_steps, input_features), return_sequences = True))
model_6.add(GRU(n_units, activation = 'relu'))
model_6.add(Dense(2, activation = 'linear'))
model_6.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_6.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 1, 16)             448       
                                                                 
 lstm_3 (LSTM)               (None, 1, 100)            46800     
                                                                 
 gru_3 (GRU)                 (None, 50)                22800     
                                                                 
 dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 70,150
Trainable params: 70,150
Non-trainable params: 0
_________________________________________________________________


In [19]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_6.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=10, batch_size = 32, verbose = 0)
    y_pred_te = model_6.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['Conv1d+LSTM+GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/23 [00:00<?, ?it/s]

22/22 [==============================] - 1s 3ms/step


  4%|▍         | 1/23 [00:10<03:48, 10.37s/it]

22/22 [==============================] - 0s 2ms/step


  9%|▊         | 2/23 [00:11<01:48,  5.15s/it]

...

 91%|█████████▏| 21/23 [01:12<00:06,  3.09s/it]

18/18 [==============================] - 0s 2ms/step


 96%|█████████▌| 22/23 [01:13<00:02,  2.63s/it]

43/43 [==============================] - 0s 3ms/step


100%|██████████| 23/23 [01:17<00:00,  3.36s/it]


### Conv1D+RNN+LSTM+GRU with tanh

In [20]:
model_8 = Sequential()
model_8.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(3, 9)))
model_8.add(SimpleRNN(n_units, activation = 'relu', return_sequences = True, input_shape = (n_steps, input_features)))
model_8.add(LSTM(200, activation = 'tanh', return_sequences = True))
model_8.add(GRU(100, activation = 'relu'))
model_8.add(Dense(2, activation = 'tanh'))
model_8.compile(optimizer='adam', loss='mae', metrics = ['mse', 'mae'])
model_8.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 1, 16)             448       
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 1, 50)             3350      
                                                                 
 lstm_4 (LSTM)               (None, 1, 200)            200800    
                                                                 
 gru_4 (GRU)                 (None, 100)               90600     
                                                                 
 dense_4 (Dense)             (None, 2)                 202       
                                                                 
Total params: 295,400
Trainable params: 295,400
Non-trainable params: 0
_________________________________________________________________


In [21]:
counter=0
for Xs_i, ys_i in tqdm(zip(ispitanci_X, ispitanici_y), total = len(ispitanici_y)):
    counter+=1
    # za sekoj ispitank "adaptiraj go model_2ot"
    X_tr, X_v, X_te = Xs_i
    y_tr, y_v, y_te = ys_i

    X_tr, y_tr = create_sequences(X_tr, y_tr, n_steps)
    X_v, y_v = create_sequences(X_v, y_v, n_steps)
    X_te, y_te = create_sequences(X_te, y_te, n_steps)

    model_8.fit(X_tr, y_tr,shuffle=False,validation_data=(X_v,y_v), epochs=20, batch_size = 32, verbose = 0)
    y_pred_te = model_8.predict(X_te)
    for metric in metric_dict:
        current_metric = metric_dict[metric]
        log_table_total.append(['tanh_Conv1D+RNN+LSTM+GRU', counter, metric,len(X_te), metric_dict[metric](y_te, y_pred_te)])

  0%|          | 0/23 [00:00<?, ?it/s]

22/22 [==============================] - 3s 6ms/step


  4%|▍         | 1/23 [00:21<07:54, 21.58s/it]

22/22 [==============================] - 0s 5ms/step


  9%|▊         | 2/23 [00:26<04:08, 11.85s/it]

...

 91%|█████████▏| 21/23 [03:32<00:17,  8.80s/it]

18/18 [==============================] - 0s 4ms/step


 96%|█████████▌| 22/23 [03:37<00:07,  7.90s/it]

43/43 [==============================] - 0s 3ms/step


100%|██████████| 23/23 [03:58<00:00, 10.38s/it]


### Log table total:

In [22]:
log_table_total = pd.DataFrame(log_table_total, columns = ['Model','Ispitanik','Metric','Length of Test','Metric score'])

In [23]:
log_table_total

,Model,Ispitanik,Metric,Length of Test,Metric score
0,DummyRegressor,1,MAE,694,0.294669
1,DummyRegressor,1,MSE,694,0.087962
2,DummyRegressor,1,R2,694,-128.700246
3,DummyRegressor,2,MAE,692,0.226303
4,DummyRegressor,2,MSE,692,0.067791
...,...,...,...,...,...
409,tanh_Conv1D+RNN+LSTM+GRU,22,MSE,571,0.067515
410,tanh_Conv1D+RNN+LSTM+GRU,22,R2,571,-9.181929
411,tanh_Conv1D+RNN+LSTM+GRU,23,MAE,1361,0.349089
412,tanh_Conv1D+RNN+LSTM+GRU,23,MSE,1361,0.140745


### Function for weighted - Mean

In [24]:
def calculate_weighted(m, log_table_total):
    num_frames = log_table_total[log_table_total['Metric'] == 'MSE']['Length of Test'].sum()
    results = []
    for d, gf in log_table_total.groupby('Metric'):
        adjusted_score = gf['Length of Test'] / num_frames * gf['Metric score']
        adjusted_score = adjusted_score.mean()
        results.append((m, d, adjusted_score))
    return results

In [25]:
final_table = []
for m, m_df in log_table_total.groupby('Model'):
    results = calculate_weighted(m, m_df)
    final_table.extend(results)

final_table_columns = ['Model', 'Metric', 'Adjusted Score']
final_table = pd.DataFrame(final_table, columns=final_table_columns)

### Final log table - Mean:

In [26]:
final_table

,Model,Metric,Adjusted Score
0,Conv1D&SimpleRNN&LSTM&GRU,MAE,0.011221
1,Conv1D&SimpleRNN&LSTM&GRU,MSE,0.003611
2,Conv1D&SimpleRNN&LSTM&GRU,R2,-1833.130601
3,Conv1d+LSTM+GRU,MAE,0.011751
4,Conv1d+LSTM+GRU,MSE,0.004279
5,Conv1d+LSTM+GRU,R2,-1630.609074
6,DummyRegressor,MAE,0.011419
7,DummyRegressor,MSE,0.003664
8,DummyRegressor,R2,-1642.487226
9,SimpleRNN+LSTM+GRU,MAE,0.011089


### Save csv file results:

In [29]:
log_table_total.to_csv('log_table_total2.csv', index=False)

In [ ]:
# from google.colab import files

# files.download('log_table_total1.csv')

In [30]:
final_table.to_csv('final_table2.csv', index=False)

In [ ]:
# from google.colab import files

# files.download('final_table1.csv')